### Search Engine with Tools and Agents

In [2]:
## Arxiv research
## Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

#### Built in Tools

In [3]:
api_wrap = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrap)
wiki.name

'wikipedia'

In [4]:
api_wrap = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrap)
arxiv.name

'arxiv'

In [5]:
tools = [wiki,arxiv]

### Custom Tools[RAG Tools]

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [11]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
docs = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(docs,OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000268557641F0>, search_kwargs={})

### Create the retriever as a Tool

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any info abt langchain")
retriever_tool.name

'langsmith-search'

In [14]:
tools = [arxiv,wiki,retriever_tool]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\mrabi\\OneDrive - Hewlett Packard Enterprise\\Documents\\DS\\Agentic AI\\Basics\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any info abt langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002684521DF30>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vector

### Run tools with Agents and LLM MOdels

In [15]:
##Tools, LLm ---> AgentExecutor
from langchain_groq import ChatGroq

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [17]:
## Prompt template

from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [18]:
## Agents - To execute tools as chains

from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm,tools,prompt)
agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [19]:
## Agent executor

from langchain.agents import AgentExecutor
agent_exe = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_exe

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [20]:
agent_exe.invoke({"input":"Tell me about Langsmith" })



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'what is Langsmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrume

{'input': 'Tell me about Langsmith',
 'output': "It looks like you've already called the tool to get some information about Langsmith. Since you've already extracted the relevant information, I can respond directly without using any more tools.\n\nLangsmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. With Langsmith, you can integrate seamlessly with LangChain's open-source frameworks and use it as a standalone platform.\n\nIt seems like you've already learned a lot about Langsmith, including how to install it, create an API key, set up your environment, log your first trace, and run your first evaluation. If you have any more questions or need further assistance, feel free to ask!"}

In [21]:
agent_exe.invoke({"input":"What is a VAE?" })



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'VAE'}`


Published: 2024-09-13
Title: VAE Explainer: Supplement Learning Variational Autoencoders with Interactive Visualization
Authors: Donald Bertucci, Alex Endert
Summary: Variational Autoencoders are widespread in Machine Learning, but are
typically explA VAE (Variational Autoencoder) is a type of generative deep learning model that is designed to learn and represent complex data distributions. It is a probabilistic approach to unsupervised learning, where the goal is to learn a probability distribution over the data.

VAEs consist of two main components: an encoder and a decoder. The encoder maps the input data to a continuous, lower-dimensional latent space, which is used to represent the data in a more abstract and compact form. The decoder maps the latent space back to the original input space, allowing the model to generate new data samples that are similar to the training data.

VAEs are often used f

{'input': 'What is a VAE?',
 'output': 'A VAE (Variational Autoencoder) is a type of generative deep learning model that is designed to learn and represent complex data distributions. It is a probabilistic approach to unsupervised learning, where the goal is to learn a probability distribution over the data.\n\nVAEs consist of two main components: an encoder and a decoder. The encoder maps the input data to a continuous, lower-dimensional latent space, which is used to represent the data in a more abstract and compact form. The decoder maps the latent space back to the original input space, allowing the model to generate new data samples that are similar to the training data.\n\nVAEs are often used for tasks such as dimensionality reduction, clustering, and generation of new data samples. They are particularly useful when dealing with high-dimensional data, where traditional methods may not be effective.\n\nThe paper "VAE Explainer: Supplement Learning Variational Autoencoders with Int